<a href="https://colab.research.google.com/github/kihongmin/DL/blob/master/toxic_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## embed_dim 150이상은 램부족..  
## complete 파일은 120_150 기준 22GB. 로컬에서 저장 비효율적

In [0]:
import numpy as np
import pandas as pd

In [0]:
from tqdm import tqdm_notebook

In [0]:
train = pd.read_csv('train.csv.zip',compression='zip')
test = pd.read_csv('test.csv.zip', compression='zip')
test_label = pd.read_csv('test_labels.csv.zip',compression='zip')

In [0]:
y = train.iloc[:,2:].values

In [0]:
train.iloc[:,2:].sum(axis=0)

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

In [0]:
import re
import nltk
import os
from nltk.tokenize import word_tokenize
from gensim.models import FastText
from nltk.corpus import stopwords
import pickle

stop_eng = stopwords.words('english')
def raw2input(raw,maxlen,embed_dim,dataClass):
    inputTitle = 'complete_'+dataClass+str(maxlen)+'_'+str(embed_dim)+'.pkl'
    preTitle = 'preprocessed_'+dataClass+'.pkl'
    dataTitle = 'tokenized_'+ dataClass +'.pkl'
    if inputTitle in os.listdir():
        with open(inputTitle, 'rb') as f:
            raw = pickle.load(f)
        return raw
    else:
        if preTitle in os.listdir():
            with open(preTitle, 'rb') as f:
                raw = pickle.load(f)
                print('complete load to preprocessed data')
        else:
            if dataTitle in os.listdir():
                with open(dataTitle, 'rb') as f: 
                    raw = pickle.load(f)
                print('complete to load data')
            else:
                raw = raw['comment_text'].apply(lambda x:x.lower())
                print('apply word_tokenize')
                raw = raw.apply(word_tokenize)
                with open(dataTitle, 'wb') as f:  
                    pickle.dump(raw, f)
            print('start to preprocess raw data')
            raw = preprocess(raw)
            with open(preTitle, 'wb') as f: 
                pickle.dump(raw, f)        
        raw = raw.to_numpy()
        ft_model = build_embedding(raw,maxlen,embed_dim,dataClass)
        for i,word_list in enumerate(raw):
            raw[i] = np.array(list(filter(lambda x: x in ft_model.wv.vocab.keys(), word_list)))

        df = np.zeros((len(raw),maxlen,embed_dim))
        print('start to fit shape')
        for i, word_list in enumerate(tqdm_notebook(raw)):
            temp = np.zeros((maxlen,embed_dim))
            index = maxlen-len(word_list) if len(word_list) < maxlen else 0
            for word in word_list:
                if index == maxlen:
                    break
                temp[index] = ft_model.wv[word]
                index+=1
            df[i] = temp
#        너무 커서 저장 안함
#        with open(inputTitle, 'wb') as f: 
#                pickle.dump(df, f,protocol=4)  
        return df
        
def build_embedding(train,maxlen,embed_dim, dataClass):
    name = 'fasttext_'+str(maxlen)+'_'+str(embed_dim)+'.vec'
    if dataClass == 'test':
        try:
            ft_model = FastText.load(name)
            return ft_model
        except:
            raise RuntimError('FastText model does not exist. Please make it by train data')
    else:        
        if name in os.listdir():
            ft_model = FastText.load(name)
            return ft_model
        else:
            print('make new FastText')
            ft_model = FastText(train, size=embed_dim, window=5, min_count=5, workers=4, sg=1)
            ft_model.save(name)
            return ft_model
    

def preprocess(df):
    df = df.apply(lambda x: list(filter(lambda y: y not in stop_eng, x)))
    p = re.compile('[A-Za-z0-9]+')
    df = df.apply(lambda x: list(filter(lambda y: p.match(y),x)))
    return df

In [0]:
maxlen=120
embed_dim = 150

In [0]:
df_train = raw2input(train,maxlen,embed_dim,'train')

complete load to preprocessed data
start to fit shape


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:45: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [0]:
df_test = raw2input(test,maxlen,embed_dim,'test')

complete load to preprocessed data
start to fit shape


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:45: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [0]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
def predict_model(df_train,y,df_test):
    inp = Input(shape=(maxlen,embed_dim, ))
    x = LSTM(60, return_sequences=True,name='lstm_layer')(inp)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(200, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    batch_size = 256
    epochs = 2
    model.fit(df_train,y,batch_size=batch_size, epochs=epochs, validation_split=0.1)
    model.save('model'+str(maxlen)+str(embed_dim)+'.h5')
    t = model.predict(df_test, batch_size=batch_size)
    sub_name = ''
    sub = pd.read_csv('sample_submission.csv.zip',compression='zip')
    subm = pd.DataFrame(t,columns=sub.columns[1:])
    subm['id'] = sub['id']
    subm.to_csv('sub4.csv',index=False)

In [0]:
predict_model(df_train,y,df_test)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 1218s 8ms/step - loss: 0.0882 - accuracy: 0.9732 - val_loss: 0.0533 - val_accuracy: 0.9805
Epoch 2/2
143613/143613 [==============================] - 1174s 8ms/step - loss: 0.0520 - accuracy: 0.9810 - val_loss: 0.0529 - val_accuracy: 0.9805
